In [ ]:
% matplotlib inline
import MySQLdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
db = MySQLdb.connect(host="crunchbase.ckfkhzm7cot3.us-west-2.rds.amazonaws.com",
                     port=3306,
                     user="root",         # your username
                     passwd="root1234",  # your password
                     db="analytics_2")        # name of the data base


In [ ]:
cur = db.cursor()

In [ ]:
count = cur.execute("SELECT count(*) FROM cb_objects")

#### Load into pandas dataframe from database

In [ ]:
df_mysql = pd.read_sql('select * from cb_objects LIMIT 100;', con=db)

In [ ]:
persons = pd.read_sql('Select * from cb_objects where entity_type = \'Person\'', con=db)

In [ ]:
relationships = pd.read_sql('Select * from cb_relationships', con=db)

In [ ]:
degrees = pd.read_sql('Select * from cb_degrees', con=db)

In [ ]:
funding_rounds = pd.read_sql('Select * from cb_funding_rounds', con=db)

#### Fill nan for empty and None values

In [ ]:
persons.fillna(value=np.nan, inplace=True)
relationships.fillna(value=np.nan, inplace=True)
degrees.fillna(value=np.nan, inplace=True)

#### People

For the baseline model we need to consider only people who are Founders, CEO, CIO etc

#### Work Experience

In [ ]:
work_ex = pd.merge(persons, relationships, how='inner', right_on='person_object_id', left_on='id')

In [ ]:
print 'Total number of work ex rows', len(work_ex)

In [59]:
workex_with_no_persons = work_ex[work_ex['id_x'].isnull()]
person_with_no_workex = work_ex[work_ex['id_y'].isnull()]
print "workex rows with no corresponding person",len(workex_with_no_persons)
print "Persons with no details about their workex",len(person_with_no_workex)

workex rows with no corresponding person 0
Persons with no details about their workex 0


In [ ]:
print "Number of rows with no start_at for workex is: ",len(work_ex[work_ex['start_at'].isnull()]), "which is roughly half the number of rows"

Idea : We can fill the start_at date with one of the following:
1. founding_date of the company
2. created_at_y which is when the record was created in the database

To analyse we can first build a plot of the different between the 2 and see the relation between the title, created at and founding date. It can mean that most of the founders have their profile created on crunchbase much later than the founding date, in which case we can safely use the founding date. However, for employess we an use the created_at.

In [ ]:
work_ex['title'].unique

This below line of code filters all the work ex where the person is one of the founding members or the board of directors

In [ ]:
founders = work_ex.loc[work_ex['id_x'].isin(work_ex.loc[work_ex['title'].str.contains('Cheif|C[A-Z]O|Co\-Founder|Co\-founder|Board', regex=True),'id_x'])]

In [55]:
founders['start_at'] = pd.to_datetime(founders['start_at'])
# import dateutil
# founderspd.to_datetime(founders['start_at'])
# founders['start_at']
# len(founders[founders['start_at'].isnull()])

/Library/Python/2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [56]:
founders.groupby('id_x').min()#agg({'start_at':min})

,entity_type,entity_id,parent_id,name,normalized_name,permalink,category_code,status,founded_at,closed_at,...,relationship_id,person_object_id,relationship_object_id,start_at,end_at,is_past,sequence,title,created_at_y,updated_at_y
id_x,,,,,,,,,,,,,,,,,,,,,
p:10,Person,10.0,NaN,Mark Zuckerberg,mark zuckerberg,/person/mark-zuckerberg,NaN,operating,NaN,NaN,...,9.0,p:10,c:5,NaN,NaN,0.0,1.0,Founder,2007-05-25 21:51:46,2010-01-25 21:49:44
p:1000,Person,1000.0,NaN,Dr. Steven E. Saunders,steven saunders,/person/dr-steven-e-saunders,NaN,operating,NaN,NaN,...,1063.0,p:1000,c:292,NaN,NaN,0.0,1.0,CTO,2007-07-26 13:31:35,2008-02-05 22:43:58
p:10000,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11190.0,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN
p:100006,Person,100006.0,NaN,Courtney Homer,courtney homer,/person/courtney-homer,NaN,operating,NaN,NaN,...,121100.0,p:100006,c:81558,2006-01-01 00:00:00,NaN,0.0,1.0,CEO & Principal Recruiter,2011-07-09 02:16:36,2012-01-18 07:34:39
p:100009,Person,100009.0,NaN,Eddy Badrina,eddy badrina,/person/eddy-badrina,NaN,operating,NaN,NaN,...,121105.0,p:100009,c:152329,2012-01-01 00:00:00,NaN,0.0,1.0,CSO and Co-Founder,2011-07-09 02:41:46,2011-07-09 12:33:00
p:100011,Person,100011.0,NaN,Jeff Grell,jeff grell,/person/jeff-grell,NaN,operating,NaN,NaN,...,121106.0,p:100011,c:43604,NaN,NaN,0.0,1.0,President and CEO,2011-07-09 03:11:26,2011-07-09 03:11:26
p:100012,Person,100012.0,NaN,Nick Bova,nick bova,/person/nick-bova,NaN,operating,NaN,NaN,...,121107.0,p:100012,c:43604,NaN,NaN,0.0,1.0,COO,2011-07-09 03:11:26,2011-07-09 03:11:26
p:100013,Person,100013.0,NaN,Umesh Singh,umesh singh,/person/umesh-singh,NaN,operating,NaN,NaN,...,121108.0,p:100013,c:3246,NaN,NaN,0.0,1.0,CFO,2011-07-09 03:11:26,2013-05-19 09:14:27
p:100014,Person,100014.0,NaN,Larry Blankenship,larry blankenship,/person/larry-blankenship,NaN,operating,NaN,NaN,...,121109.0,p:100014,c:81561,NaN,NaN,0.0,1.0,Chairman of the Board and CEO,2011-07-09 03:17:16,2011-07-09 03:17:16


In [58]:
founder_groups = founders.groupby('id_x')#.agg({'start_at':pd.datetime})
founder_groups['start_at'].min()

id_x
p:10                 NaN
p:1000               NaN
p:10000     1.275350e+18
p:100006    1.136074e+18
p:100009    1.325376e+18
p:100011             NaN
p:100012             NaN
p:100013             NaN
p:100014             NaN
p:100015             NaN
p:100016             NaN
p:100017    1.104538e+18
p:100024    1.044058e+18
p:100025    1.044058e+18
p:100026    1.044058e+18
p:100029             NaN
p:100030             NaN
p:100031    1.197590e+18
p:100046    1.262304e+18
p:100047    1.283299e+18
p:100048             NaN
p:100049    9.783072e+17
p:100062    1.136074e+18
p:100065    8.520768e+17
p:100071    1.293840e+18
p:100072    1.167610e+18
p:100073    1.104538e+18
p:100074    1.272672e+18
p:100079             NaN
p:10008     1.199146e+18
                ...     
p:99931     1.296518e+18
p:99932     1.296518e+18
p:99937     1.201824e+18
p:99939     1.293840e+18
p:99940     1.293840e+18
p:99943     1.302134e+18
p:9995               NaN
p:99952              NaN
p:99955             

In [53]:
import datetime
# pd.to_datetime(founders['start_at'])[5] > pd.to_datetime('2012-01-01')
np.min(pd.to_datetime(founders['start_at']))

Timestamp('1960-01-01 00:00:00')

TODO: Aggregate as the min of teh start_at, this will give the first date when work ex added

#### Education

In [ ]:
len(degrees)

In [ ]:
degrees[:5]

In [ ]:
len(degrees['object_id'].unique())

We have degree details for the above number of people

We need to give a rating to the degrees. All Bachelors will have rating as 1. Masters will have rating 2 and PHD/MBA will have rating 3.As there are around 7000 differnet degrees, for other types of degrees we can use the median of the rest of the table. 

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('B\..*'),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('Bachelor'),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('BS'),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('JD'),'degree_score'] = 1

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('Master'),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('MS'),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('M.$'),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('M\..*'),'degree_score'] = 2

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('Ph\.*D'),'degree_score'] = 3

In [ ]:
degrees.loc[degrees['degree_type'].str.contains('MBA'),'degree_score'] = 3

In [ ]:
degrees.loc[degrees['degree_type'] == '','degree_score'] = 1

In [ ]:
degrees['degree_score'].fillna(np.median(degrees['degree_score']), inplace=True)

In [ ]:
degrees[['degree_type','degree_score']][:5]

In [ ]:
degrees.loc[degrees['degree_score'].isnull()]['degree_type'].describe()

In [ ]:
education = pd.merge(persons, degrees, how='inner', left_on='id', right_on='object_id')

In [ ]:
education_with_no_persons = education[education['id_x'].isnull()]
person_with_no_education = education[education['id_y'].isnull()]
print "Education rows with no corresponding person",len(education_with_no_persons)
print "Persons with no details about their education",len(person_with_no_education)

Next Steps: 
1. We have sparse data for education. What to do with it?
2. How to clean workex data?

#### Funding Rounds

In [ ]:
len(funding_rounds)

In [ ]:
work_ex_funding = pd.merge(work_ex, funding_rounds,how='inner', left_on='relationship_object_id', right_on='object_id')

In [ ]:
len(work_ex_funding)

In [ ]:
work_ex_funding[:5]
funding_rounds['funded_at_datetime']= pd.to_datetime(funding_rounds['funded_at'])
funding_rounds[['funded_at_datetime','funded_at']]

In [ ]:
kevin = work_ex_funding[work_ex_funding['id_x']=='p:3']
kevin[['title','funded_at']]
kevin

In [ ]:
kevin1 = work_ex[work_ex['id_x']=='p:3']
# kevin1[['title','funded_at']]
# kevin1['start_at_datetime'] = pd.to_datetime(kevin['start_at'].dropna(),yearfirst = True)
# pd.to_datetime(kevin['end_at'])
# pd.to_datetime(kevin['end_at'],infer_datetime_format=True)
# kevin1[['start_at', 'end_at','start_at_datetime', 'end_at_datetime']]
# kevin1[['start_at', 'end_at']]
# pd.to_datetime(kevin1['start_at'].dropna())

In [ ]:
a = '2001-02-01'
pd.to_datetime(a)

In [ ]:
a = work_ex[:5].groupby('id_x')
work_ex[:10]

### Train Test and Evaluation Data

We need to divide the data into train test and evaluation data. We will divide it in the following ratio:

1.Train - 60%

2.Test - 20%

3.Evaluation - 20%


### Baseline Model

Our baseline model is dependent on Education and work experience of the founder. We will train our linear regression model on the score of the founder based on the above 2 features and run it on the test data. 

We will then find the error rate of our baseline model on the test data.